In [79]:
import os
os.chdir('//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/libprad')
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/COMBINATIONS/'

import numpy as np
import json
import itertools
import requests
import importlib
from tqdm import tqdm

import logbook_api
import webapi_access as api
import dat_lists as lists
import radiational_fraction as radfrac
import tomogram_metrics as tm
import training as training
import correlation as corr
import sensitivity as sense
import training_plot as tpl
import plot_funcs as pf
import prad_calculation as prad_calc
import mClass

import matplotlib.pyplot as p
import matplotlib as mpl
from matplotlib import cm

import training
import sensitivity
import correlation
import training_plot

info = lists.geom_dat_to_json()
vbc_list = info['channels']['eChannels']['VBC']
hbc_list = info['channels']['eChannels']['HBCm']

In [80]:
info = lists.geom_dat_to_json()

dirs = next(os.walk(loc))[1]
# dirs.remove('bak')
dirs.remove('old')

methods = next(os.walk(loc + dirs[0] + '/'))[1][::-1]
combs = next(os.walk(loc + dirs[0] +
             '/' + methods[0] + '/'))[1]

In [ ]:
importlib.reload(training)
importlib.reload(sensitivity)
importlib.reload(correlation)
importlib.reload(training_plot)

foo = training.training_master(
    program='20181010.032',
    URI_vers='V4', mode='weighted_deviation',
    nCH=3, saving=True, plot=True)

In [111]:
importlib.reload(pf)
importlib.reload(corr)
importlib.reload(sense)
importlib.reload(tpl)
importlib.reload(prad_calc)
importlib.reload(mClass)

base_URI = 'http://archive-webapi.ipp-hgw.mpg.de/'
dat_URI = base_URI + 'Test/raw/W7XAnalysis/QSB_Bolometry/'
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/COMBINATIONS/'

iterator = list(itertools.product(
    ['20181010.032'],
    ['self_correlation'],
    ['3_HBCm']))
#     methods, combs))  # dirs, ['coherence_fft'], combs))[:1]  

for i in tqdm(range(len(iterator)), desc='combinations'):
    entry = iterator[i]
    # for i, entry in enumerate(iterator):
    xpid, method, comb = entry
    if i == 0:
        xpid_old = xpid
        # print('\\\ ', xpid, end=' ')

    if True:  # try:
        # load file of comb and method for xpid
        # print('>> loading file:', method, comb)
        file = loc + xpid + '/' + method + \
            '/' + comb + '/spectrum_analysis_' + method + '.json'
        with open(file, 'r') as f:
            training_results = json.load(f)
        f.close()

        N = training_results['combination']
        # best/worst combination/cells
        ind = np.argmax(training_results['correlation'])
        best_comb = training_results['combinations'][ind]  # best single combination
        ind = np.argmin(training_results['correlation'])
        worst_comb = training_results['combinations'][ind]  # worst single combination

        best_cell = np.argsort(np.nan_to_num(training_results['av_sense_channels']))[-N:]
        best1 = [training_results['av_sense_channels'][i]
                for i in best_cell]  # best indiv. channels by hits
        worst_cell = np.argsort(training_results['av_sense_channels'])[:N]
        worst = [training_results['av_sense_channels'][i]
                for i in worst_cell]  # worst indiv. channels by hits

        best_cell2 = np.argsort(np.nan_to_num(training_results['av_sense_combination']))[-N:]
        best2 = [training_results['av_sense_combination'][i]
                for i in best_cell2]  # best indiv. channels by combinations
        arr = np.array(training_results['av_sense_combination']).astype('float')
        arr[arr == .0] = np.nan
        worst_cell2 = np.argsort(arr)[:N]
        worst2 = [arr[i] for i in worst_cell2]  # worst indiv. channels by combinations

        # stuff to do once per experiment
        if i == 0 or xpid_old != xpid:
            # print('\t\\\ download data ...')
            dat = {}
            program_info = api.xpid_info(program=xpid)[0]
            dat['BoloSignal'] = api.download_single(
                data_req=dat_URI + 'BoloAdjusted_DATASTREAM/V4/',
                program_info=program_info, debug=False)
            dat['P_rad_hbc'] = api.download_single(
                data_req=dat_URI + 'PradHBCm_DATASTREAM/V4/0/PradHBCm/',
                program_info=program_info, debug=False)
            dat['power'] = api.download_single(
                data_req=dat_URI + 'PradChannels_DATASTREAM/V4/',
                program_info=program_info, debug=False)

            # print('\t\\\ priority data')
            prio, status = api.do_before_running(
                program_info=program_info, program=xpid,
                date=xpid[0:8], data_object=dat)
            # need stop and start indices
            t_1, t_2 = (prio['t0'] + 1e8), (prio['t4'] - 1e8)
            ix_s = mClass.find_nearest(np.array(
                dat['BoloSignal']['dimensions']), t_1)[0]
            ix_t = mClass.find_nearest(np.array(
                dat['BoloSignal']['dimensions']), t_2)[0]
            if i == 0:
                broken_channels = prio['geometry']['channels']['droplist']
            
            # print('\t\\\ default values')
            alpha, phi, beta = corr.default_correlation(
                method=method,
                arr1=dat['P_rad_hbc']['values'],
                arr2=dat['P_rad_hbc']['values'],
                id_1=ix_s, id_2=ix_t,
                time_raw=dat['BoloSignal']['dimensions'])
            # print('\t\\\ calculate for program')

        for combination, name in zip(  # different N channel combinations
                [best_cell, best_comb],  # worst_comb, worst_cell, best_cell2, worst_cell2],
                ['best_chans', 'best_comb'  # , 'worst_comb', 'worst_chans',
                # 'best_chans_comb', 'worst_chans_comb'
                ]):
            calculation, volume_sum = prad_calc.calculate_prad(
                power=np.array(dat['power']['values'], dtype=np.float),
                volume=prio['geometry']['geometry']['vbolo'],
                k_bolo=prio['geometry']['geometry']['kbolo'],
                volume_torus=prio['volume_torus'],
                channels=combination,
                camera_list=combination,
                date=int(xpid[0:8]),
                shotno=int(xpid[-3:]),
                brk_chan=broken_channels,
                debug=False)

            freqs = np.zeros((500))
            e_val, trace, freqs, pref, lab, ylab = corr.evaluation(
                method=method,
                arr1=np.array(dat['P_rad_hbc']['values']),
                arr2=calculation,
                id_1=ix_s, id_2=ix_t,
                time_raw=dat['BoloSignal']['dimensions'],
                freqs=freqs,
                plot=False)
            theta = e_val / alpha if method is not \
                'mean_deviation' else e_val
            # print('\t\t\\\ ', name, '...', combination, ':', theta)

            # plot best combination
            fig_location = '../results/COMBINATIONS/' + xpid + '/' + name + '_'
            tpl.correlation_comb_plot(
                arr1=np.array(dat['P_rad_hbc']['values'], dtype=np.float),
                arr2=calculation, time=np.array(prio['time']),
                method=method, pref=pref, lab=lab, ylab=ylab, loc=fig_location,
                cell=combination,
                trace=trace, phi=phi, freqs=freqs, beta=1.,  # beta,
                alpha=alpha, theta=theta,
                id_1=ix_s, id_2=ix_t)
            # break
            # break

    # except Exception as e:
    #     print('error', xpid, method, comb)
    #     pass

    if i != 0:
        if xpid_old != xpid:
            xpid_old = xpid
            print('\\\ ', xpid, end=' ')

combinations: 100%|██████████| 1/1 [00:11<00:00, 11.59s/it]


In [115]:
fig, ax = p.subplots()
ax.plot(np.nan_to_num(training_results['correlation']), c='k')
ax.set_xlim(0, len(training_results['correlation']))
ax.set_xlabel('combination [#]')
ax.set_ylabel('quality [a.u.]')
fig.set_size_inches(5., 2.5)
fig.savefig(method + '_3_HBCm_combinations.pdf',
            dpi=169.0, bbox_inches='tight')
p.close('all')

In [ ]:
importlib.reload(pf)
importlib.reload(corr)
importlib.reload(sense)
importlib.reload(tpl)
importlib.reload(prad_calc)
importlib.reload(mClass)

base_URI = 'http://archive-webapi.ipp-hgw.mpg.de/'
dat_URI = base_URI + 'Test/raw/W7XAnalysis/QSB_Bolometry/'
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/COMBINATIONS/'

colors = cm.brg(np.linspace(.0, 1., 6))

iterator = list(itertools.product(
    dirs, # ['20180725.044', '20180809.013'],
    methods, # ['coherence_fft'],
    combs))  # dirs, methods, combs))

for j in tqdm(range(len(iterator)), desc='combinations'):
    # for j, entry in enumerate(iterator):
    entry = iterator[j]
    xpid, method, comb = entry
    # print(entry)

    if j == 0:
        xpid_old = xpid
        method_old = method
        res_list = []
        # print('\\\ ', xpid, end=' ')
        # print('\\\ ', method, end=' ')

    # if True:
    try:
        # load file of comb and method for xpid
        # print('>> loading file:', method, comb)
        file = loc + xpid + '/' + method + \
            '/' + comb + '/spectrum_analysis_' + method + '.json'
        with open(file, 'r') as f:
            training_results = json.load(f)
        f.close()
        res_list.append(np.array([
            training_results['combinations'],
            training_results['correlation']
        ]))

        if len(res_list) == 6:
            fig = p.figure()
            ax = fig.add_subplot(111)
            fig_location = '../results/COMBINATIONS/' + xpid_old + '/'

            for i, foo in enumerate(res_list):
                N = 30  # int(np.shape(foo)[1] / 30.)
                X = np.linspace(.0, 1., np.shape(foo)[1])
                # X = np.linspace(1., np.shape(foo)[1] - 1, np.shape(foo)[1])
                M, cam = combs[i].replace('_', ' ').split()
                ax.plot(
                    np.convolve(X, np.ones((N, )) / N, mode='valid'),
                    np.convolve(foo[1], np.ones((N, )) / N, mode='valid'),
                    c=colors[i], alpha=.75, label=cam + ', ' + M)

            ax.set_yticklabels([''] * 10)
            ax.set_xticklabels([''] * 10)

            ax.set_xlim(.0, 1.)
            ax.set_ylabel('quality [a.u.]')
            ax.set_xlabel('combination [#]')
            ax.legend(
                loc='upper center', bbox_to_anchor=(.5, 1.3),
                ncol=3, fancybox=True, shadow=False,
                handletextpad=0.3, labelspacing=0.5, handlelength=1.)

            fig.set_size_inches(5., 3.)
            fig.savefig('../results/CURRENT/combinations_methods.pdf')
            fig.savefig(
                fig_location + method + '_' +
                xpid_old.replace('.', '_') + '_combinations.pdf')
            res_list = []
            p.close('all')

    except Exception as e:
        print('error', xpid, method, comb)
        pass

    if j != 0:
        if xpid_old != xpid:
            xpid_old = xpid
            # print('\\\ ', xpid, end=' ')
        if method_old != method:
            method_old = method
            # print('\\\ ', method, end=' ')

In [ ]:
importlib.reload(pf)
importlib.reload(corr)
importlib.reload(sense)
importlib.reload(tpl)
importlib.reload(prad_calc)
importlib.reload(mClass)

base_URI = 'http://archive-webapi.ipp-hgw.mpg.de/'
dat_URI = base_URI + 'Test/raw/W7XAnalysis/QSB_Bolometry/'
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/COMBINATIONS/'

colors = cm.brg(np.linspace(.0, 1., 3))
markers = ['x', '^', 's']

iterator = list(itertools.product(
    ['20180725.044'],  # dirs, # ['20180725.044', '20180809.013'],
    ['weighted_deviation'], # methods, # ['coherence_fft'],
    ['3_HBCm', '5_HBCm', '7_HBCm']))

for j in tqdm(range(len(iterator)), desc='combinations'):
    # for j, entry in enumerate(iterator):
    entry = iterator[j]
    xpid, method, comb = entry

    if j == 0:
        xpid_old = xpid
        method_old = method
        res_list = []
        # print('\\\ ', xpid, end=' ')
        # print('\\\ ', method, end=' ')

    if True:
        # try:
        # load file of comb and method for xpid
        # print('>> loading file:', method, comb)
        file = loc + xpid + '/' + method + \
            '/' + comb + '/spectrum_analysis_' + method + '.json'
        with open(file, 'r') as f:
            training_results = json.load(f)
        f.close()
        res_list.append(np.array([
            training_results['av_sense_channels'],
            training_results['av_sense_combination']
        ]))

        if len(res_list) == 3:
            fig = p.figure()
            ax = fig.add_subplot(111)
            fig_location = '../results/COMBINATIONS/' + xpid_old + '/'

            for i, foo in enumerate(res_list):
                M, cam = ['3_HBCm', '5_HBCm', '7_HBCm'][i].replace('_', ' ').split()
                ax.plot(foo[0], c=colors[i], alpha=.75, ls='-.',
                        marker=markers[i], label=cam + ', ' + M)
            
            ax.set_xlim(.0, 31.)
            ax.set_ylabel('quality [a.u.]')
            ax.set_xlabel('channel [#]')
            ax.legend(
                loc='upper center', bbox_to_anchor=(.5, 1.3),
                ncol=3, fancybox=True, shadow=False,
                handletextpad=0.3, labelspacing=0.5, handlelength=1.)

            fig.set_size_inches(5., 3.)
            fig.savefig('../results/CURRENT/sensitivity_method.pdf')
            fig.savefig(
                fig_location + method + '_' +
                xpid_old.replace('.', '_') + '_sensitivity_combs.pdf')
            res_list = []
            p.close('all')

    # except Exception as e:
    #     print('error', xpid, method, comb)
    #     pass

    if j != 0:
        if xpid_old != xpid:
            xpid_old = xpid
            # print('\\\ ', xpid, end=' ')
        if method_old != method:
            method_old = method
            # print('\\\ ', method, end=' ')

In [39]:
methods

['weighted_deviation',
 'self_correlation',
 'mean_deviation',
 'fft_solo',
 'fft_and_convolve',
 'fftconvolve_integrated',
 'correlation',
 'coherence_fft']

In [78]:
importlib.reload(pf)
importlib.reload(corr)
importlib.reload(sense)
importlib.reload(tpl)
importlib.reload(prad_calc)
importlib.reload(mClass)

base_URI = 'http://archive-webapi.ipp-hgw.mpg.de/'
dat_URI = base_URI + 'Test/raw/W7XAnalysis/QSB_Bolometry/'
loc = '//share.ipp-hgw.mpg.de/documents/pih/Documents/git/QSB_Bolometry/results/COMBINATIONS/'

minor_radius = 0.5139

colors = cm.brg(np.linspace(.0, 1., 3))
markers = ['x', '^', 's']

iterator = list(itertools.product(
    ['weighted_deviation'],  # ['weighted_deviation'],
    # ['3_HBCm', '5_HBCm', '7_HBCm'],  # combs,
    ['3_VBC', '5_VBC', '7_VBC'],  # combs,
    dirs))  # ['20180725.044']))

loc_max = .0
res_list, fin_list, errors = [], [], []
for j in tqdm(range(len(iterator)), desc='combinations'):
    # for j, entry in enumerate(iterator):
    entry = iterator[j]
    method, comb, xpid = entry

    if True:
        # try:
        # load file of comb and method for xpid
        # print('>> loading file:', method, comb)
        file = loc + xpid + '/' + method + \
            '/' + comb + '/spectrum_analysis_' + method + '.json'
        with open(file, 'r') as f:
            training_results = json.load(f)
        f.close()
        res_list.append(np.array([
            training_results['av_sense_channels'],
            training_results['av_sense_combination']
        ]))

        if len(res_list) == 52:
            res_list = np.array(res_list)
            foo = res_list.reshape(2, 52, 128)
            res_list = []

            bar = np.percentile(foo[1], 97.5, axis=(0))
            fin_list.append(bar)
            loc_max = np.nansum(bar) / np.shape(np.unique(
                bar))[0] if np.nansum(bar) / np.shape(np.unique(
                    bar))[0] > loc_max else loc_max

            E = np.zeros((128))
            for c in range(128):
                a, b = \
                    np.percentile(foo[1, :, c], 85.), \
                    np.percentile(foo[1, :, c], 97.5)
                E[c] = np.std(foo[1, :, c][
                    np.where((a < foo[1, :, c]) & (foo[1, :, c] < b))[0]])
            errors.append(E)

        if len(fin_list) == 3:
            fig = p.figure()
            ax = fig.add_subplot(111)
            ar = ax.twiny()

            k = 0
            for X, E in zip(fin_list, errors):
                # M, cam = ['3_HBCm', '5_HBCm', '7_HBCm'][k].replace(
                #     '_', ' ').split()
                M, cam = ['3_VBC', '5_VBC', '7_VBC'][k].replace(
                    '_', ' ').split()
            
                no_nans = np.argwhere(X == X)
                channels = [x for x in np.unique(no_nans)]
                NChannels = np.linspace(
                    .0, len(channels) - 1, len(channels))

                x = X[channels]
                e = E[channels]

                R = []
                for c, channel in enumerate(channels):
                    R.append(info['radius']['reff'][channel] / minor_radius)

                index_list = np.argsort(np.array(R))
                sort_channels = np.array(channels)[index_list]
                sort_R = np.array(R)[index_list]

                if ('fft' in method) or (method == 'correlation'):
                    X = X / loc_max
                    E = E / loc_max

                ax.errorbar(
                    np.linspace(0, 127, 128), X, yerr=E,
                    # NChannels, x, yerr=e,
                    c=colors[k], alpha=.75, ls='-.',
                    marker=markers[k], label=cam + ', ' + M)
                k += 1
            fin_list = []
            errors = []

            for axis in [ax, ar]:
                axis.set_xlim(np.min(channels), np.max(channels))
            tick_locs = ax.get_xticks()
            N = int(np.ceil(
                np.shape(channels)[0] / np.shape(tick_locs)[0]))

            # ar.set_xticks(channels[::N])
            ar.set_xticklabels([str(round(r, 2)) for r in sort_R[::N]])
            ar.set_xlabel('radius [r$_{a}$]')

            ax.set_ylabel('quality [a.u.]')
            ax.set_xlabel('channel [#]')
            ax.legend()
            #     loc='upper center', bbox_to_anchor=(.5, 1.2),
            #     ncol=3, fancybox=True, shadow=False,
            #     handletextpad=0.3, labelspacing=0.5, handlelength=1.)

            fig.set_size_inches(5., 3.)
            fig.savefig('../results/CURRENT/sensitivity_combs.pdf')
            fig_location = '../results/COMBINATIONS/'
            fig.savefig(
                fig_location + method + '_sensitivity_combs_' + cam + '.pdf')
            p.close('all')

    # except Exception as e:
    #     print('error', xpid, method, comb)
    #     pass

combinations: 100%|██████████| 156/156 [00:05<00:00, 28.51it/s]


In [ ]:
full_results = {
    'xpids': dirs,
    'data': {
    }
}

xpids = np.array(dirs)
iterator = list(itertools.product(
    methods,  # ['weighted_deviation'],
    combs,
    dirs))  # [:100]
for i in tqdm(range(len(iterator)), desc='combinations'):
    method, comb, xpid = iterator[i]
    if i == 0:
        old_comb, old_method = comb, method

    if method != old_method or i == 0:
        full_results['data'][method] = {}

    if comb != old_comb or i == 0:
        full_results['data'][method][comb] = {
            'av_sense_channels': None,
            'av_sense_combination': None}
        av_channels = np.zeros((128, len(dirs)))
        av_combination = np.zeros((128, len(dirs)))

    file = loc + xpid + '/' + method + \
        '/' + comb + '/spectrum_analysis_' + method + '.json'
    with open(file, 'r') as f:
        dat = json.load(f)
    f.close()

    j = np.where(xpids == xpid)[0][0]
    av_channels[:, j] = dat['av_sense_channels']
    av_combination[:, j] = dat['av_sense_combination']

    if ((i != len(iterator) - 1) and (
            comb != iterator[i + 1][1])) or (
            i == len(iterator) - 1):
        full_results['data'][method][comb][
            'av_sense_channels'] = av_channels
        full_results['data'][method][comb][
            'av_sense_combination'] = av_combination

    if i != 0:
        if method != old_method:
            old_method = method
        if comb != old_comb:
            old_comb = comb

np.save(
    '../results/COMBINATIONS/training_results.npy',
    full_results)

In [ ]:
full_results = np.load('../results/COMBINATIONS/training_results.npy', allow_pickle=True).item()

In [ ]:
ecrh_power = []
ne = []
Te_core = []
Te_out = []
prad = []
frad = []

for i in tqdm(range(len(full_results['xpids'])), desc='programs'):
    program = full_results['xpids'][i]
    program_info, req = api.xpid_info(program=program)
    t0 = program_info['programs'][0]['trigger']['1'][0]  # in ns
    t4 = program_info['programs'][0]['trigger']['4'][0]  # in ns

    fr, rad, ecrh = radfrac.radiational_frac(program=program)

    try:
        ecrh_power.append(np.array(ecrh['values']))
    except Exception as e:
        ne.append(np.array([np.nan]))

    try:
        prad.append(np.array(rad['values']))
        if np.max(prad[-1]) > 10.:
            prad.append(np.array([np.nan]))
    except Exception as e:
        prad.append(np.array([np.nan]))

    try:
        frad.append(np.array(fr[1])[
            ~np.isnan(np.array(fr[1]))])
    except Exception as e:
        frad.append(np.array([np.nan]))

    try:
        ne.append(np.array(api.download_single(
            api.download_link(name='n_e lint'),
            program_info=None, filter=None,
            start_POSIX=t0, stop_POSIX=t4)['values']))
    except Exception:
        ne.append(np.array([np.nan]))

    try:
        Te_core.append(np.array(api.download_single(
            api.download_link(name='T_e ECE core'),
            program_info=None, filter=None,
            start_POSIX=t0, stop_POSIX=t4)['values']))
    except Exception as e:
        Te_core.append(np.array([np.nan]))

    try:
        Te_out.append(np.array(api.download_single(
            api.download_link(name='T_e ECE out'),
            program_info=None, filter=None,
            start_POSIX=t0, stop_POSIX=t4)['values']))
    except Exception as e:
        Te_out.append(np.array([np.nan]))

proginfo_training = {
    'ECRH': ecrh_power,
    'ne lint': ne,
    'Te core': Te_core,
    'Te out': Te_out,
    'prad': prad,
    'frad': frad}
np.save('../results/COMBINATIONS/proginfo_training_results.npy', proginfo_training)

In [ ]:
proginfo_training = np.load('../results/COMBINATIONS/proginfo_training_results.npy', allow_pickle=True).item()

In [ ]:
entry = 'av_sense_channels'
cams = ['HBCm', 'VBC']
param_list = [
    ['ECRH', 'ECRH [MW]', 1.],
    ['ne lint', 'n$_{e}$ int. [10$^{19}$ m$^{-3}$]', 1.e19],
    ['Te core', 'T$_{e}$ [keV]', 1.],
    ['Te out', 'T$_{e}$ [keV]', 1.],
    ['prad', 'P$_{rad}$ [MW]', 1.],
    ['frad', 'f$_{rad}$ [a.u.]', 1.]]
iterator = list(itertools.product(cams, methods, param_list))

for j in tqdm(range(len(iterator)), desc='combinations'):
    cam, method, param_entry = iterator[j]
    param, cbarax, f = param_entry

    channels = info['channels']['eChannels'][cam]
    camrange = [np.min(channels), np.max(channels)]
    dataM = full_results['data'][method]

    full_sense = []
    for comb in combs:
        if (cam not in comb) or (
                isinstance(dataM[comb][entry], None.__class__)):
            continue

        for trainingXP in dataM[comb][entry].transpose():
            full_sense.append(trainingXP)
    full_sense = np.array(full_sense).transpose()

    no_nans = np.argwhere(full_sense == full_sense)
    channels = [x for x in np.unique(no_nans[:, 0])]

    N = 25  # number of bins
    bins_y  = np.linspace(
        np.min(np.nan_to_num(full_sense, np.inf)),
        np.max(np.nan_to_num(full_sense, -np.inf)),
        N)

    # XP data, devide as well
    parameter = proginfo_training[param]

    contour = np.zeros((N, np.shape(channels)[0]))
    ticks = np.zeros((N, np.shape(channels)[0]))
    for c, channel in enumerate(channels):
        diggi = np.digitize(full_sense[channel], bins_y)

        for xpid in range(N):
            biny = diggi[xpid] - 1
            if (np.shape(parameter[biny])[0]) > 0 and (
                    parameter[biny][0] != .0) and (~np.isnan(parameter[biny][0])):
                contour[biny, c] += np.quantile(
                    parameter[biny] / f, .95)
                ticks[biny, c] += 1.

    fig = p.figure()
    ax = fig.add_subplot()
    ar = ax.twiny()

    minor_radius = 0.5139
    NChannels = np.linspace(
        .0, len(channels) - 1, len(channels))

    R = []
    for c, channel in enumerate(channels):
        R.append(info['radius']['reff'][channel] / minor_radius)

    index_list = np.argsort(np.array(R))
    sort_channels = np.array(channels)[index_list]
    sort_contour = contour[:, index_list]
    sort_ticks = ticks[:, index_list]
    sort_R = np.array(R)[index_list]

    for axis in [ax, ar]:
        # axis.set_xlim(np.min(channels), np.max(channels))
        axis.set_xlim(np.min(NChannels), np.max(NChannels))

    # xx, yy = np.meshgrid(channels, bins_y)
    xx, yy = np.meshgrid(NChannels, bins_y)
    cs = ax.contourf(
        xx, yy,
        np.nan_to_num(contour / ticks),
        20, cmap='plasma',
        alpha=1.)

    cax = fig.add_axes([0.98, 0.13, 0.03, 0.74])
    cbar = fig.colorbar(cs, orientation='vertical', cax=cax)

    tick_locs = ax.get_xticks()
    N = int(np.ceil(
        # np.shape(channels)[0] / np.shape(tick_locs)[0]))
        np.shape(NChannels)[0] / np.shape(tick_locs)[0]))

    # ar.set_xticks(channels[::N])
    ar.set_xticks(NChannels[::N])
    # ar.set_xticklabels([str(round(r, 2)) for r in R[::N]])
    ar.set_xticklabels([str(round(r, 2)) for r in sort_R[::N]])

    ar.set_xlabel('radius [r$_{a}$]')
    ax.set_xlabel('channel [#]')
    ax.set_ylabel('quality [a.u.]')
    cbar.ax.set_ylabel(cbarax)

    fig.tight_layout()
    fig.set_size_inches(5., 3.)

    pf.fig_current_save('sensitivity_parameters', fig)
    fig_location = '../results/COMBINATIONS/' + method + \
        '_sensitivity_params_' + param.replace(' ', '_') + '_' + cam + '.pdf'
    fig.savefig(fig_location)
    p.close('all')

In [ ]:
fig = p.figure()
ax = fig.add_subplot()
ar = ax.twiny()

minor_radius = 0.5139
NChannels = np.linspace(
    .0, len(channels) - 1, len(channels))

R = []
for c, channel in enumerate(channels):
    R.append(info['radius']['reff'][channel] / minor_radius)

index_list = np.argsort(np.array(R))
sort_channels = np.array(channels)[index_list]
sort_contour = contour[:, index_list]
sort_ticks = ticks[:, index_list]
sort_R = np.array(R)[index_list]

for axis in [ax, ar]:
    # axis.set_xlim(np.min(channels), np.max(channels))
    axis.set_xlim(np.min(NChannels), np.max(NChannels))

# xx, yy = np.meshgrid(channels, bins_y)
xx, yy = np.meshgrid(NChannels, bins_y)
cs = ax.contourf(
    xx, yy,
    np.nan_to_num(contour / ticks),
    20, cmap='plasma',
    alpha=1.)

cax = fig.add_axes([0.98, 0.13, 0.03, 0.74])
cbar = fig.colorbar(cs, orientation='vertical', cax=cax)

tick_locs = ax.get_xticks()
N = int(np.ceil(
    # np.shape(channels)[0] / np.shape(tick_locs)[0]))
    np.shape(NChannels)[0] / np.shape(tick_locs)[0]))

# ar.set_xticks(channels[::N])
ar.set_xticks(NChannels[::N])
# ar.set_xticklabels([str(round(r, 2)) for r in R[::N]])
ar.set_xticklabels([str(round(r, 2)) for r in sort_R[::N]])

ar.set_xlabel('radius [r$_{a}$]')
ax.set_xlabel('channel [#]')
ax.set_ylabel('quality [a.u.]')
cbar.ax.set_ylabel(cbarax)

fig.tight_layout()
fig.set_size_inches(5., 3.)

pf.fig_current_save('sensitivity_parameters', fig)
fig_location = '../results/COMBINATIONS/' + method + \
    '_sensitivity_params_' + param.replace(' ', '_') + '_' + cam + '.pdf'
fig.savefig(fig_location)